In [1]:
import numpy as np
import pandas as pd
import os


d:\Tools\anaconda3\envs\pytorch-py38\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### 1、读取数据
读取训练数据，测试数据以及停用词

In [2]:
train=pd.read_csv(r'./data/train.tsv.zip',sep='\t')
test=pd.read_csv(r'./data/test.tsv.zip',sep='\t')
with open(r'./data/stopwords.txt',encoding='utf8')as f:
    stopwords=f.read().split('\n')
train.head(2)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2


In [62]:
phrase_list=train.Phrase.to_list()
sentiment_list=train.Sentiment.to_list()
phrase_list=[phrase_list[i].lower().split() for i in range(len(phrase_list))]
phrase_list2=test.Phrase.to_list()
phrase_list2=[phrase_list2[i].lower().split() for i in range(len(phrase_list2))]


### 2、处理数据
去除停用词
记录词频，将字符转为向量

In [63]:
for p in range(len(phrase_list)):
    new_list=[]
    for w in phrase_list[p]:
        word=w.strip("',?.!^%￥#@~·&*（）(`)_-+=:;\|")
        if word not in stopwords:
            if len(word)>1:
                new_list.append(word)
    phrase_list[p]=new_list


In [64]:
for p in range(len(phrase_list2)):
    new_list=[]
    for w in phrase_list2[p]:
        word=w.strip("',?.!^%￥#@~·&*（）(`)_-+=:;\|")
        if word not in stopwords:
            if len(word)>1:
                new_list.append(word)
    phrase_list2[p]=new_list

In [65]:
from collections import defaultdict
word_dict=defaultdict(int)
for i in phrase_list:
    for j in i:
        word_dict[j]+=1
sorted_word_dict = sorted(word_dict.items(), key = lambda kv:(kv[1], kv[0]),reverse=True)

In [66]:
freq=[]
n=300
for i in range(n):
    freq.append(sorted_word_dict[i][0])

In [67]:
X=[]
for i in phrase_list:
    x=[0]*n
    for j in i:
        if j in freq:
            x[freq.index(j)]+=1
    X.append(x)
X=np.array(X)

In [82]:
X_test=[]
for i in phrase_list2:
    xt=[0]*n
    for j in i:
        if j in freq:
            xt[freq.index(j)]+=1
    X_test.append(xt)
X_test=np.array(X_test)

In [85]:
y=np.array(sentiment_list)

### 3、模型训练
将数据输入模型进行训练

In [91]:
from sklearn.neural_network import MLPClassifier
rs=42
clf=MLPClassifier().fit(X,y)
clf.score(X, y)

d:\Tools\anaconda3\envs\pytorch-py38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


0.614962194027938

In [92]:
# 预测
predict=clf.predict(X_test)

In [95]:
# 保存模型
import pickle
with open(r'./model/MLPClf.pkl','wb')as f:
    pickle.dump(clf,f)
with open(r'./model/MLPClf.pkl','rb')as f:
    clf=pickle.load(f)

In [96]:
clf.score(X, y)

0.614962194027938